In [1]:
!pip install webdriver_manager

In [15]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.parse import urlparse
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/maiapolo/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
/usr/local/anaconda/envs/detectshift/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [3]:
links = []

for i in tqdm(range(1,21)):
    
    driver.get("https://www.president.gov.ua/en/news/speeches?page="+str(i))

    source=str(driver.page_source)
    txt = source
    ind, ind2, ind3 = 0, 0, 0

    while True:
        txt=txt[(ind+ind2+ind3):]

        ind=txt.find('item_stat_headline')
        ind2=txt[ind:].find('https://')
        ind3=txt[(ind+ind2):].find('">')

        links.append([txt[(ind+ind2):(ind+ind2+ind3)]])

        if links[-1]==['']: 
            links=links[:-1]
            break

100%|██████████| 20/20 [00:09<00:00,  2.06it/s]


In [4]:
len(links)

192

In [5]:
for i in tqdm(range(len(links))):

    driver.get(links[i][0])

    source=str(driver.page_source)
    txt = source
    
    ind=txt.rfind('"date"')+7
    ind2=txt[ind:].find('</p>')
    date=txt[ind:(ind+ind2)]
    date=" ".join(date.split())

    ind3=source[(ind+ind2):].find('title="" alt="')+14
    ind4=source[(ind+ind2+ind3):].find('">')
    title=source[(ind+ind2+ind3):(ind+ind2+ind3+ind4)]

    soup = bs(source[(ind+ind2):], features="html.parser")
    text = soup.get_text()
    ind5=text.rfind('\n\n\n\n\n\nv')
    text = text[:ind5]
    text=text.replace("\n", " ")
    text=" ".join(text.split())

    links[i]+=[date,title,text]

100%|██████████| 192/192 [04:52<00:00,  1.52s/it]


In [12]:
data = {'url':[l[0] for l in links], 'date':[l[1] for l in links], 'title':[l[2] for l in links], 'text':[l[3] for l in links]}

In [17]:
data=pd.DataFrame(data)

In [19]:
data.to_csv('data/zelensky_speeches.csv')